<a href="https://colab.research.google.com/github/nithinrk11/FlowCast/blob/main/Necessary_script_files.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Combined code cell of the ML models

In [ ]:
!pip install tensorflow scikit-learn
!git clone https://github.com/nithinrk11/FlowCast.git
import numpy as np

#decision tree
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import pickle

#random forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import label_binarize

#XGboost
from sklearn.preprocessing import label_binarize
from xgboost import XGBClassifier

#XGboost Hyperparameter
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier

#DNN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

#Models
dec = DecisionTreeClassifier()
rf = RandomForestClassifier()
xg = XGBClassifier()
best_xgb = XGBClassifier()
model = Sequential()



# Load the noisy dataset
data = pd.read_csv('/content/FlowCast/noisy_crowd_data2.csv', parse_dates=['Timestamp'])

# Encode the 'Crowd_Type' column
label_encoder = LabelEncoder()
data['Crowd_Type_Label'] = label_encoder.fit_transform(data['Crowd_Type'])

# Define features and target
X = data[['Noisy_Crowd_Count']]
y = data['Crowd_Type_Label']


# Decision tree
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the decision tree model
dec = DecisionTreeClassifier(random_state=42)

# Train the model
dec.fit(X_train, y_train)

# Make predictions on the test set
y_pred = dec.predict(X_test)

# Calculate metrics
#accuracy = accuracy_score(y_test, y_pred)
#conf_matrix = confusion_matrix(y_test, y_pred)
#class_report = classification_report(y_test, y_pred)

#print(f"Decision tree Accuracy: {accuracy:.2f}")
#print("\nDecision tree Confusion Matrix:")
#print(conf_matrix)
#print("\nDecision tree Classification Report:")
#print(class_report)
#-----------------------------------------------------------------------------#

# random forest
# Binarize the output for multiclass classification
y_bin = label_binarize(y, classes=np.unique(y))


# Create a Random Forest classifier
rf = RandomForestClassifier(n_estimators=50, random_state=42)

# Train the model
rf.fit(X_train, y_train)

# Make predictions on the test set
y_pred_rf = rf.predict(X_test)

# Evaluate accuracy
#accuracy_rf = accuracy_score(y_test, y_pred_rf)
#print(f"Random Forest Accuracy: {accuracy_rf:.2f}")

# Confusion Matrix
#conf_matrix_rf = confusion_matrix(y_test, y_pred_rf)
#print("\nRandom Forest Confusion Matrix:")
#print(conf_matrix_rf)

# Classification Report
#class_report_rf = classification_report(y_test, y_pred_rf)
#print("\nRandom Forest Classification Report:")
#print(class_report_rf)
#----------------------------------------------------------------------------#
# XGBoost
# Set the number of classes explicitly
num_classes = len(np.unique(y))

# Create an XGBoost classifier with num_class parameter
xg = XGBClassifier(num_class=num_classes)

# Train the model
xg.fit(X_train, y_train)

# Make predictions on the test set
y_pred = xg.predict(X_test)

# Evaluate accuracy
#accuracy = accuracy_score(y_test, y_pred)
#print(f"XGBoost Accuracy: {accuracy:.2f}")

# Confusion Matrix
#conf_matrix = confusion_matrix(y_test, y_pred)
#print("\nXGBoost Confusion Matrix:")
#print(conf_matrix)

# Classification Report
#class_report = classification_report(y_test, y_pred)
#print("\nXGBoost Classification Report:")
#print(class_report)
#---------------------------------------------------------------------------#

# XGBoost Hyperparameter
# Define the parameter grid for hyperparameter tuning
param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'n_estimators': [50, 100, 200],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
}

# Create an XGBoost classifier
xgb = XGBClassifier(random_state=42)

# Perform grid search
grid_search = GridSearchCV(estimator=xgb, param_grid=param_grid, scoring='accuracy', cv=5)
grid_search.fit(X_train, y_train)

# Get the best parameters
best_params = grid_search.best_params_
print(f"Best Hyperparameters: {best_params}")

# Use the best model for prediction
best_xgb = grid_search.best_estimator_
y_pred = best_xgb.predict(X_test)

# Evaluate accuracy
#accuracy = accuracy_score(y_test, y_pred)
#print(f"Tuned XGBoost Accuracy with Best Model: {accuracy:.2f}")

# Confusion Matrix
#conf_matrix = confusion_matrix(y_test, y_pred)
#print("\nTuned XGBoost Confusion Matrix:")
#print(conf_matrix)

# Classification Report
#class_report = classification_report(y_test, y_pred)
#print("\nTuned XGBoost Classification Report:")
#print(class_report)
#----------------------------------------------------------------------------#

# DNN
# Set random seed for NumPy
np.random.seed(42)

# Convert labels to one-hot encoding
y_train_one_hot = to_categorical(y_train)
y_test_one_hot = to_categorical(y_test)

# Build the DNN model

model.add(Dense(128, input_dim=1, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(3, activation='softmax'))  # Adjusted output layer for three classes

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train_one_hot, epochs=20, batch_size=32, validation_split=0.2)

# Evaluate the model on the test set
y_pred = model.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)
y_test_labels = np.argmax(y_test_one_hot, axis=1)

# Calculate metrics
#accuracy = accuracy_score(y_test_labels, y_pred_labels)
#conf_matrix = confusion_matrix(y_test_labels, y_pred_labels)
#class_report = classification_report(y_test_labels, y_pred_labels)

#print(f"DNN Accuracy: {accuracy:.2f}")
#print("\nDNN Confusion Matrix:")
#print(conf_matrix)
#print("\nDNN Classification Report:")
#print(class_report)

#Dumping the models into correspondind pickle file in write mode
pickle.dump(dec,open('dec.pkl','wb'))
pickle.dump(rf,open('rf.pkl','wb'))
pickle.dump(xg,open('xg.pkl','wb'))
pickle.dump(best_xgb,open('best_xgb.pkl','wb'))
pickle.dump(model,open('model.pkl','wb'))





Cloning into 'FlowCast'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 16 (delta 3), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (16/16), 196.65 KiB | 762.00 KiB/s, done.
Resolving deltas: 100% (3/3), done.
Best Hyperparameters: {'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 200, 'subsample': 1.0}
Epoch 1/20
175/175 [==============================] - 7s 6ms/step - loss: 1.0141 - accuracy: 0.3715 - val_loss: 0.9146 - val_accuracy: 0.3807
Epoch 2/20
175/175 [==============================] - 1s 5ms/step - loss: 0.7355 - accuracy: 0.6323 - val_loss: 0.3420 - val_accuracy: 0.9450
Epoch 3/20
175/175 [==============================] - 1s 5ms/step - loss: 0.2472 - accuracy: 0.9032 - val_loss: 0.1698 - val_accuracy: 0.9379
Epoch 4/20
175/175 [==============================] - 1s 5ms/step - loss: 0.2012 - accuracy: 0.9086 - val_loss: 0.1899 - va

#Streamlit code
* The below is the most simplified edit and first stage of web-app design edit where the data input is done by a single crowd count number from the slider option by the user.
* `New updated code edits are being done and will be updated further below.`

In [ ]:
import streamlit as st
import pickle
import numpy as np

dec = pickle.load(open('dec.pkl', 'rb'))
rf = pickle.load(open('rf.pkl', 'rb'))
xg = pickle.load(open('xg.pkl', 'rb'))
best_xgb = pickle.load(open('best_xgb.pkl', 'rb'))
model = pickle.load(open('model.pkl', 'rb'))

def classify(num, model):
    # Reshape the input to a 2D array
    inputs = np.array([num]).reshape(1, -1)

    if num <= 30:
        return "Low Crowd"
    elif 30 < num <= 70:
        return "Moderate Crowd"
    else:
        return "High Crowd"

def main():
    st.title("Streamlit Tutorial")
    html_temp = """
    <div style="background-color:teal ;padding:10px">
    <h2 style="color:white;text-align:center;">Crowd Classification</h2>
    </div>
    """
    st.markdown(html_temp, unsafe_allow_html=True)
    activities = ['Decision tree', 'Random forest', 'XGBoost', 'Tuned XGBoost', 'DNN']
    option = st.sidebar.selectbox('Which model would you like to use?', activities)
    st.subheader(option)
    sl = st.slider('Crowd count', 10, 150)
    if st.button('Classify'):
        st.success(classify(sl, dec) if option == 'Decision tree' else
                  classify(sl, rf) if option == 'Random forest' else
                  classify(sl, xg) if option == 'XGBoost' else
                  classify(sl, best_xgb) if option == 'Tuned XGBoost' else
                  classify(sl, model))

if __name__ == '__main__':
    main()


#UPDATES

#Updated Streamlit code with improved informative detailing and input data is uploaded as CSV file for classification
* Further improvements are expected to be done in future untill final decision of edit

In [ ]:
import streamlit as st
import pandas as pd
import pickle
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load the models
dec = pickle.load(open('dec.pkl', 'rb'))
rf = pickle.load(open('rf.pkl', 'rb'))
xg = pickle.load(open('xg.pkl', 'rb'))
best_xgb = pickle.load(open('best_xgb.pkl', 'rb'))
model = pickle.load(open('model.pkl', 'rb'))

def classify(num, model):
    if num <= 30:
        return "Low Crowd"
    elif 30 < num <= 70:
        return "Moderate Crowd"
    else:
        return "High Crowd"

def classify_range(crowd_counts, model):
    results = [classify(num, model) for num in crowd_counts]
    return results

def main():
    st.title("Project FlowCast")
    html_temp = """
    <div style="background-color:teal ;padding:10px">
    <h2 style="color:white;text-align:center;">Crowd Classification</h2>
    </div>
    """
    st.markdown(html_temp, unsafe_allow_html=True)
    activities = ['Decision tree', 'Random forest', 'XGBoost', 'Tuned XGBoost', 'DNN']
    option = st.sidebar.selectbox('Which model would you like to use?', activities)
    st.subheader(option)

    uploaded_file = st.file_uploader("Upload CSV file", type=["csv"])

    if uploaded_file is not None:
        data = pd.read_csv(uploaded_file)

        # Assuming the column containing crowd counts is named 'Crowd_Count'
        if 'Crowd_Count' in data.columns and 'Timestamp' in data.columns:
            data['Timestamp'] = pd.to_datetime(data['Timestamp'])

            # Resample the data for daily, weekly, and monthly averages
            daily_avg = data.resample('D', on='Timestamp').mean()
            weekly_avg = data.resample('W-Mon', on='Timestamp').mean()
            monthly_avg = data.resample('M', on='Timestamp').mean()

            crowd_counts = data['Crowd_Count'].tolist()
            timestamps = data['Timestamp']

            results = classify_range(crowd_counts, dec) if option == 'Decision tree' else \
                      classify_range(crowd_counts, rf) if option == 'Random forest' else \
                      classify_range(crowd_counts, xg) if option == 'XGBoost' else \
                      classify_range(crowd_counts, best_xgb) if option == 'Tuned XGBoost' else \
                      classify_range(crowd_counts, model)

            # Calculate overall average crowd count
            overall_avg = np.mean(crowd_counts)

            # Display the time versus crowd count graph with different colors for each crowd type
            fig, ax = plt.subplots(figsize=(10, 6))
            sns.lineplot(x=timestamps, y=crowd_counts, hue=results, palette="viridis", ax=ax, label='Crowd Count')

            # Plot average trend lines
            sns.lineplot(x=daily_avg.index, y=daily_avg['Crowd_Count'], color='blue', label='Daily Avg', ax=ax)
            sns.lineplot(x=weekly_avg.index, y=weekly_avg['Crowd_Count'], color='green', label='Weekly Avg', ax=ax)
            sns.lineplot(x=monthly_avg.index, y=monthly_avg['Crowd_Count'], color='orange', label='Monthly Avg', ax=ax)

            # Plot overall average line
            plt.axhline(overall_avg, color='red', linestyle='--', label='Overall Avg')

            plt.title('Time vs Crowd Count with Average Estimation')
            plt.xlabel('Timestamp')
            plt.ylabel('Crowd Count')
            plt.legend()
            st.pyplot(fig)

            # Display the crowd count classifications in a table
            results_df = pd.DataFrame({'Timestamp': timestamps, 'Crowd_Count': crowd_counts, 'Classification': results})
            st.write(results_df)
        else:
            st.error("Please make sure your CSV file has columns named 'Timestamp' and 'Crowd_Count'.")

if __name__ == '__main__':
    main()
